In [1]:
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

Modell ermitteln: https://huggingface.co/spaces/mteb/leaderboard

In [19]:
#model_name = "jinaai/jina-embeddings-v3"                 # am schnellsten
#model_name = "intfloat/multilingual-e5-large-instruct"   # etwas langsamer
model_name = "Snowflake/snowflake-arctic-embed-l-v2.0"    # ca. genau so schnell
#model_name = "infly/inf-retriever-v1-1.5b"               # etwa 3x langsamer

model = SentenceTransformer(model_name, trust_remote_code=True).cuda()

In [3]:
import json
import lzma
with lzma.open("llm-abstract-sentences.json.xz", "rt") as f:
    es = json.loads(f.read())

In [8]:
len(es)

270409

In [20]:
%%time
# Test zur Geschwindigkeitsmessung
embeddings = model.encode([e["title"] + ": " + e["text"] for e in es[0:1000]])

CPU times: user 5.13 s, sys: 56.8 ms, total: 5.18 s
Wall time: 4.17 s


In [21]:
embeddings.shape

(1000, 1024)

In [22]:
# alle Embeddings ausrechnen
embeddings = model.encode([e["title"] + ": " + e["text"] for e in es])

In [23]:
import numpy as np
with open("llm-abstract-sentences-saev2.npy", "wb") as f:
    np.save(f, embeddings)

In [27]:
qe = model.encode("How long do I train an LLM?", prompt_name="query")

In [28]:
similarities = model.similarity(qe, embeddings).flatten().numpy()

In [29]:
for i in similarities.argsort()[::-1][0:5]:
    print(es[i])

{'id': '2309.03852', 'title': 'FLM-101B: An Open LLM and How to Train It with $100K Budget', 'text': 'FLM-101B: An Open LLM and How to Train It with $100K Budget. Large language models (LLMs) are considered important approaches towards foundational machine intelligence, achieving remarkable success in Natural Language Processing and multimodal tasks, among others. However, the carbon footprints and financial costs originating from heavy pre-training computation is a non-negligible issue. Progressive training methods, inspired by the neurogenesis process that grows neural structures, have shown potential to accelerate LLM pre-training. However, the algorithms, implementation, and practices for progressively training LLMs beyond 100B parameters remain underexplored.'}
{'id': '2404.14395', 'title': 'PARAMANU-GANITA: Language Model with Mathematical Capabilities', 'text': 'Paramanu-Ganita took 146 hours of A100 training whereas math specialised LLM, LLEMMA 7B, was trained for 23,000 A100 h